<a href="https://colab.research.google.com/github/TheHoyer/MEDICA/blob/main/Medica_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interfejs HTML do dodawania nowego pacjenta i zapisywanie danych do bazy

In [1]:
# -*- coding: utf-8 -*-
from IPython.display import display, HTML
import sqlite3
import hashlib
import os
from google.colab import drive, output
import re
import json

DRIVE_MOUNT_POINT = '/content/drive'
DB_FOLDER_PATH = os.path.join(DRIVE_MOUNT_POINT, 'MyDrive', 'ColabPatientData')
DB_FILE_PATH = os.path.join(DB_FOLDER_PATH, 'patients.db')

try:
    drive.mount(DRIVE_MOUNT_POINT, force_remount=True)
    print(f"Dysk Google zamontowany w {DRIVE_MOUNT_POINT}")
    os.makedirs(DB_FOLDER_PATH, exist_ok=True)
    print(f"Folder bazy danych pacjentów: {DB_FOLDER_PATH}")
except Exception as e:
    print(f"Błąd montowania Dysku Google: {e}")
    raise SystemExit("Nie można zamontować Dysku Google. Sprawdź uprawnienia.")

def init_patient_db():
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS patients (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                imie TEXT NOT NULL,
                nazwisko TEXT NOT NULL,
                pesel TEXT UNIQUE NOT NULL,
                data_urodzenia TEXT,
                telefon TEXT,
                email TEXT UNIQUE NOT NULL,
                adres TEXT,
                kod_pocztowy TEXT,
                miasto TEXT,
                kontakt_ice_imie_nazwisko TEXT,
                kontakt_ice_relacja TEXT,
                kontakt_ice_telefon TEXT,
                alergie TEXT,
                choroby_przewlekle TEXT,
                wazne_uwagi TEXT,
                password_hash TEXT NOT NULL
            )
        ''')
        conn.commit()
        print(f"Baza danych pacjentów zainicjalizowana lub już istnieje w {DB_FILE_PATH}")
    except sqlite3.Error as e:
        print(f"Błąd inicjalizacji bazy danych pacjentów: {e}")
    finally:
        if conn:
            conn.close()

def hash_password(password):
    return hashlib.sha256(password.encode('utf-8')).hexdigest()

def is_valid_email(email):
    if not email: return False
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return re.match(pattern, email) is not None

def is_valid_pesel(pesel):
    if not pesel: return False
    return re.match(r"^\d{11}$", pesel) is not None

def zarejestruj_pacjenta(
    imie, nazwisko, pesel, data_urodzenia, telefon, email,
    adres, kod_pocztowy, miasto,
    kontakt_ice_imie_nazwisko, kontakt_ice_relacja, kontakt_ice_telefon,
    alergie, choroby_przewlekle, wazne_uwagi, password):

    if not all([imie, nazwisko, pesel, email, password]):
        # Mimo że JS zignoruje, nadal zwracamy błąd dla logiki Pythona
        return json.dumps({'status': 'error', 'message': 'Imię, nazwisko, PESEL, email i hasło są wymagane.'})
    if not is_valid_pesel(pesel):
        return json.dumps({'status': 'error', 'message': 'Nieprawidłowy format PESEL (wymagane 11 cyfr).'})
    if not is_valid_email(email):
        return json.dumps({'status': 'error', 'message': 'Nieprawidłowy format email.'})
    if len(password) < 8:
        return json.dumps({'status': 'error', 'message': 'Hasło musi mieć co najmniej 8 znaków.'})

    hashed_pass = hash_password(password)
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE_PATH)
        cursor = conn.cursor()
        cursor.execute('''
            INSERT INTO patients (
                imie, nazwisko, pesel, data_urodzenia, telefon, email,
                adres, kod_pocztowy, miasto,
                kontakt_ice_imie_nazwisko, kontakt_ice_relacja, kontakt_ice_telefon,
                alergie, choroby_przewlekle, wazne_uwagi, password_hash
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            imie, nazwisko, pesel, data_urodzenia, telefon, email,
            adres, kod_pocztowy, miasto,
            kontakt_ice_imie_nazwisko, kontakt_ice_relacja, kontakt_ice_telefon,
            alergie, choroby_przewlekle, wazne_uwagi, hashed_pass
        ))
        conn.commit()
        # Ten print nadal będzie widoczny pod komórką Colab
        print(f"Zarejestrowano pacjenta: {imie} {nazwisko} (PESEL: {pesel})")
        # Zwracamy sukces, mimo że JS może mieć problem z odczytem
        return json.dumps({'status': 'success', 'message': f'Pacjent {imie} {nazwisko} został pomyślnie zarejestrowany.'})
    except sqlite3.IntegrityError as e:
        error_msg = str(e).lower()
        if 'patients.pesel' in error_msg:
            print(f"Błąd: Pacjent z PESEL '{pesel}' już istnieje.")
            return json.dumps({'status': 'error', 'message': f'Pacjent z numerem PESEL {pesel} już istnieje w bazie.'})
        elif 'patients.email' in error_msg:
             print(f"Błąd: Pacjent z email '{email}' już istnieje.")
             return json.dumps({'status': 'error', 'message': f'Pacjent z adresem email {email} już istnieje w bazie.'})
        else:
            print(f"Błąd unikalności: {e}")
            return json.dumps({'status': 'error', 'message': f'Błąd unikalności danych: {e}'})
    except sqlite3.Error as e:
        print(f"Błąd zapisu pacjenta do bazy danych: {e}")
        if "malformed" not in str(e).lower():
             return json.dumps({'status': 'error', 'message': f'Błąd zapisu do bazy danych: {e}'})
        else:
             print("KRYTYCZNY BŁĄD: Baza danych jest uszkodzona (malformed). Proszę usunąć plik patients.db z Dysku Google i spróbować ponownie.")
             return json.dumps({'status': 'error', 'message': 'KRYTYCZNY BŁĄD: Baza danych jest uszkodzona. Skontaktuj się z administratorem lub usuń plik patients.db i spróbuj ponownie.'})
    finally:
        if conn:
            conn.close()

output.register_callback('registerPatientCallback', zarejestruj_pacjenta)

init_patient_db()

print("\nŚrodowisko Python gotowe. Wyświetlanie formularza rejestracji pacjenta...")

html_content = """
<!DOCTYPE html>
<html lang="pl">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Rejestracja Nowego Pacjenta</title>
<style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap');
    :root{--primary-blue:#007bff;--light-blue:#e7f3ff;--text-color:#333;--border-color:#ced4da;--background-color:#f8f9fa;--white:#ffffff;--gray:#6c757d;--light-gray:#adb5bd;--link-color:#007bff;--danger-color:#dc3545;--success-color:#28a745;}
    body { background-color: var(--background-color); font-family: 'Roboto', sans-serif; margin: 0; padding: 20px;}
    .form-container { background-color: var(--white); max-width: 800px; margin: 20px auto; padding: 30px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); }
    .form-container h1 { text-align: center; color: var(--primary-blue); margin-bottom: 30px; font-size: 1.8rem; font-weight: 700;}
    .form-section { margin-bottom: 30px; border-bottom: 1px solid var(--border-color); padding-bottom: 20px; }
    .form-section:last-child { border-bottom: none; margin-bottom: 0; padding-bottom: 0; }
    .form-section h2 { color: var(--primary-blue); font-size: 1.2rem; margin-bottom: 15px; border-bottom: 2px solid var(--primary-blue); padding-bottom: 5px; display: inline-block; }
    .form-grid { display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px; }
    .form-group { margin-bottom: 15px; }
    .form-group label { display: block; margin-bottom: 6px; font-weight: 500; color: var(--text-color); font-size: 0.9rem; }
    .form-group input[type="text"],
    .form-group input[type="email"],
    .form-group input[type="tel"],
    .form-group input[type="date"],
    .form-group input[type="password"],
    .form-group textarea { width: 100%; padding: 10px; border: 1px solid var(--border-color); border-radius: 5px; font-size: 1rem; box-sizing: border-box; }
    .form-group textarea { min-height: 80px; resize: vertical; }
    .form-group input:focus, .form-group textarea:focus { outline: none; border-color: var(--primary-blue); box-shadow: 0 0 0 2px rgba(0,123,255,0.25); }
    .btn { display: inline-block; width: 100%; padding: 12px; border: none; border-radius: 5px; font-size: 1.1rem; font-weight: 500; cursor: pointer; text-align: center; transition: background-color 0.3s ease, box-shadow 0.3s ease; box-sizing: border-box; background-color: var(--primary-blue); color: var(--white); margin-top: 20px; }
    .btn:disabled { background-color: var(--light-gray); cursor: not-allowed; }
    .btn:hover:not(:disabled) { background-color: #0056b3; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
    #status-message { margin-top: 20px; font-weight: 500; min-height: 1.2em; text-align: center; transition: color 0.3s ease, background-color 0.3s ease, border 0.3s ease; padding: 10px; border-radius: 5px; border: 1px solid transparent; }
    .status-success { color: var(--success-color); background-color: #d4edda; border-color: #c3e6cb; }
    .status-error { color: var(--danger-color); background-color: #f8d7da; border-color: #f5c6cb;}
    .status-processing { color: var(--gray); background-color: #e2e3e5; border-color: #d6d8db;}
    @media (max-width: 600px) {
        .form-grid { grid-template-columns: 1fr; }
        .form-container { padding: 20px; }
        .form-container h1 { font-size: 1.5rem; }
    }
</style>
</head>
<body>

<div class="form-container">
    <h1>Dodaj Nowego Pacjenta</h1>
    <form id="register-patient-form">

        <div class="form-section">
            <h2>Dane Podstawowe</h2>
            <div class="form-grid">
                <div class="form-group"><label for="imie">Imię <span style="color:red;">*</span></label><input type="text" id="imie" name="imie" required></div>
                <div class="form-group"><label for="nazwisko">Nazwisko <span style="color:red;">*</span></label><input type="text" id="nazwisko" name="nazwisko" required></div>
                <div class="form-group"><label for="pesel">PESEL <span style="color:red;">*</span></label><input type="text" id="pesel" name="pesel" required maxlength="11" pattern="\d{11}" title="Podaj 11 cyfr"></div>
                <div class="form-group"><label for="data_urodzenia">Data Urodzenia</label><input type="date" id="data_urodzenia" name="data_urodzenia"></div>
            </div>
        </div>

        <div class="form-section">
            <h2>Dane Kontaktowe</h2>
            <div class="form-grid">
                <div class="form-group"><label for="telefon">Numer Telefonu</label><input type="tel" id="telefon" name="telefon"></div>
                <div class="form-group"><label for="email">Adres Email <span style="color:red;">*</span></label><input type="email" id="email" name="email" required></div>
                 <div class="form-group"><label for="adres">Adres Zamieszkania</label><input type="text" id="adres" name="adres"></div>
                 <div class="form-group"><label for="kod_pocztowy">Kod Pocztowy</label><input type="text" id="kod_pocztowy" name="kod_pocztowy"></div>
                 <div class="form-group"><label for="miasto">Miasto</label><input type="text" id="miasto" name="miasto"></div>
            </div>
        </div>

         <div class="form-section">
            <h2>Kontakt ICE (w nagłym wypadku)</h2>
            <div class="form-grid">
                <div class="form-group"><label for="kontakt_ice_imie_nazwisko">Imię i Nazwisko Kontaktu</label><input type="text" id="kontakt_ice_imie_nazwisko" name="kontakt_ice_imie_nazwisko"></div>
                 <div class="form-group"><label for="kontakt_ice_relacja">Relacja</label><input type="text" id="kontakt_ice_relacja" name="kontakt_ice_relacja"></div>
                <div class="form-group"><label for="kontakt_ice_telefon">Numer Telefonu Kontaktu</label><input type="tel" id="kontakt_ice_telefon" name="kontakt_ice_telefon"></div>
            </div>
        </div>

        <div class="form-section">
             <h2>Informacje Medyczne</h2>
             <div class="form-grid">
                 <div class="form-group"><label for="alergie">Alergie</label><textarea id="alergie" name="alergie" placeholder="Wpisz znane alergie..."></textarea></div>
                 <div class="form-group"><label for="choroby_przewlekle">Choroby Przewlekłe</label><textarea id="choroby_przewlekle" name="choroby_przewlekle" placeholder="np. Nadciśnienie tętnicze, Cukrzyca typu 2..."></textarea></div>
                 <div class="form-group"><label for="wazne_uwagi">Ważne uwagi medyczne</label><textarea id="wazne_uwagi" name="wazne_uwagi"></textarea></div>
             </div>
        </div>

         <div class="form-section">
            <h2>Dane Logowania (dla Pacjenta)</h2>
             <p style="font-size: 0.85rem; color: var(--gray);">Te dane będą używane przez pacjenta do logowania się do systemu.</p>
            <div class="form-grid">
                <div class="form-group"><label for="password">Hasło <span style="color:red;">*</span></label><input type="password" id="password" name="password" required minlength="8"></div>
                <div class="form-group"><label for="confirm_password">Potwierdź Hasło <span style="color:red;">*</span></label><input type="password" id="confirm_password" name="confirm_password" required minlength="8"></div>
            </div>
        </div>

        <button type="submit" id="register-button" class="btn">Zarejestruj Pacjenta</button>
        <div id="status-message"></div>

    </form>
</div>

<script>
    const form = document.getElementById('register-patient-form');
    const statusMessageDiv = document.getElementById('status-message');
    const registerButton = document.getElementById('register-button');
    const imieInput = document.getElementById('imie');
    const nazwiskoInput = document.getElementById('nazwisko');
    const peselInput = document.getElementById('pesel');
    const dataUrodzeniaInput = document.getElementById('data_urodzenia');
    const telefonInput = document.getElementById('telefon');
    const emailInput = document.getElementById('email');
    const adresInput = document.getElementById('adres');
    const kodPocztowyInput = document.getElementById('kod_pocztowy');
    const miastoInput = document.getElementById('miasto');
    const iceImieNazwiskoInput = document.getElementById('kontakt_ice_imie_nazwisko');
    const iceRelacjaInput = document.getElementById('kontakt_ice_relacja');
    const iceTelefonInput = document.getElementById('kontakt_ice_telefon');
    const alergieInput = document.getElementById('alergie');
    const chorobyInput = document.getElementById('choroby_przewlekle');
    const uwagiInput = document.getElementById('wazne_uwagi');
    const passwordInput = document.getElementById('password');
    const confirmPasswordInput = document.getElementById('confirm_password');

    const emailRegex = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
    const peselRegex = /^\d{11}$/;

    function showStatus(message, type) {
        statusMessageDiv.textContent = message;
        statusMessageDiv.className = `status-${type}`;
    }

    form.addEventListener('submit', async function(event) {
        event.preventDefault();
        registerButton.disabled = true;
        showStatus('Sprawdzanie danych...', 'processing');

        const imie = imieInput.value.trim();
        const nazwisko = nazwiskoInput.value.trim();
        const pesel = peselInput.value.trim();
        const data_urodzenia = dataUrodzeniaInput.value;
        const telefon = telefonInput.value.trim();
        const email = emailInput.value.trim();
        const adres = adresInput.value.trim();
        const kod_pocztowy = kodPocztowyInput.value.trim();
        const miasto = miastoInput.value.trim();
        const kontakt_ice_imie_nazwisko = iceImieNazwiskoInput.value.trim();
        const kontakt_ice_relacja = iceRelacjaInput.value.trim();
        const kontakt_ice_telefon = iceTelefonInput.value.trim();
        const alergie = alergieInput.value.trim();
        const choroby_przewlekle = chorobyInput.value.trim();
        const wazne_uwagi = uwagiInput.value.trim();
        const password = passwordInput.value;
        const confirmPassword = confirmPasswordInput.value;

        if (!imie || !nazwisko || !pesel || !email || !password || !confirmPassword) {
            showStatus('Pola oznaczone * (Imię, Nazwisko, PESEL, Email, Hasło, Potwierdź Hasło) są wymagane.', 'error');
            registerButton.disabled = false; return;
        }
        if (!peselRegex.test(pesel)) {
            showStatus('Nieprawidłowy format PESEL (wymagane 11 cyfr).', 'error');
            registerButton.disabled = false; return;
        }
        if (!emailRegex.test(email)) {
            showStatus('Nieprawidłowy format adresu email.', 'error');
            registerButton.disabled = false; return;
        }
        if (password.length < 8) {
            showStatus('Hasło musi mieć co najmniej 8 znaków.', 'error');
            registerButton.disabled = false; return;
        }
        if (password !== confirmPassword) {
            showStatus('Hasła nie są zgodne.', 'error');
            registerButton.disabled = false; return;
        }

        showStatus('Rejestrowanie pacjenta...', 'processing');

        try {
            // Wywołaj funkcję Pythona, ignorując szczegóły zwracanego obiektu 'result'
            await google.colab.kernel.invokeFunction(
                'registerPatientCallback',
                [
                    imie, nazwisko, pesel, data_urodzenia, telefon, email,
                    adres, kod_pocztowy, miasto,
                    kontakt_ice_imie_nazwisko, kontakt_ice_relacja, kontakt_ice_telefon,
                    alergie, choroby_przewlekle, wazne_uwagi, password
                ],
                {}
            );

            // Załóż sukces po stronie Pythona (bo print pod komórką się pojawia)
            // i wyświetl komunikat w formularzu
            const successMessage = `Pacjent ${imie} ${nazwisko} (PESEL: ${pesel}) został pomyślnie zarejestrowany.`;
            showStatus(successMessage, 'success');
            form.reset(); // Wyczyść formularz

        } catch (error) {
            // Ten błąd wystąpi tylko, jeśli samo wywołanie invokeFunction się nie powiedzie
            console.error('Błąd wywołania funkcji Pythona (registerPatient):', error);
            showStatus('Wystąpił błąd komunikacji z backendem podczas rejestracji.', 'error');
            // W tym przypadku nie czyścimy formularza
        } finally {
             // Zawsze odblokuj przycisk na koniec
             registerButton.disabled = false;
        }
    });

</script>

</body>
</html>
"""

display(HTML(html_content))

Mounted at /content/drive
Dysk Google zamontowany w /content/drive
Folder bazy danych pacjentów: /content/drive/MyDrive/ColabPatientData
Baza danych pacjentów zainicjalizowana lub już istnieje w /content/drive/MyDrive/ColabPatientData/patients.db

Środowisko Python gotowe. Wyświetlanie formularza rejestracji pacjenta...


# Logowanie pacjenta do systemu

In [ ]:
# -*- coding: utf-8 -*-
# Blok 2: Logowanie Pacjenta (Ustawia stan zalogowania)

from IPython.display import display, HTML
import sqlite3
import hashlib
import os
from google.colab import drive, output
import re
import json
from datetime import datetime

logged_in_patient_data = None

DRIVE_MOUNT_POINT = '/content/drive'
DB_FOLDER_PATH = os.path.join(DRIVE_MOUNT_POINT, 'MyDrive', 'ColabPatientData')
DB_FILE_PATH = os.path.join(DB_FOLDER_PATH, 'patients.db')

if not os.path.exists(DRIVE_MOUNT_POINT + '/MyDrive'):
    print("BŁĄD: Dysk Google nie jest zamontowany. Uruchom najpierw komórkę montującą.")
if not os.path.exists(DB_FILE_PATH):
     print(f"OSTRZEŻENIE: Plik bazy danych {DB_FILE_PATH} nie istnieje. Czy uruchomiono blok rejestracji?")

def hash_password(password):
    return hashlib.sha256(password.encode('utf-8')).hexdigest()

def is_valid_email(email):
    if not email: return False
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return re.match(pattern, email) is not None

def is_valid_pesel(pesel):
    if not pesel: return False
    return re.match(r"^\d{11}$", pesel) is not None

def calculate_age(birthdate_str):
    if not birthdate_str: return None
    try:
        birthdate = datetime.strptime(birthdate_str, '%Y-%m-%d').date()
        today = datetime.today().date()
        age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
        return age
    except ValueError: return None

def zaloguj_pacjenta(identifier, password):
    global logged_in_patient_data
    logged_in_patient_data = None

    if not identifier or not password:
        return json.dumps({'status': 'error', 'message': 'Email/PESEL i hasło są wymagane.'})

    hashed_input_password = hash_password(password)
    conn = None
    try:
        conn = sqlite3.connect(DB_FILE_PATH)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()

        is_email = is_valid_email(identifier)
        is_pesel = is_valid_pesel(identifier)

        select_query = "SELECT * FROM patients WHERE "
        if is_email: select_query += "email = ?"
        elif is_pesel: select_query += "pesel = ?"
        else: return json.dumps({'status': 'error', 'message': 'Nieprawidłowy format identyfikatora (oczekiwano email lub PESEL).'})

        cursor.execute(select_query, (identifier,))
        patient_data_row = cursor.fetchone()

        if patient_data_row:
            stored_hash = patient_data_row['password_hash']
            if hashed_input_password == stored_hash:
                print(f"Pomyślnie zalogowano pacjenta: {patient_data_row['imie']} {patient_data_row['nazwisko']} (ID: {patient_data_row['id']})")
                logged_in_patient_data = dict(patient_data_row)
                logged_in_patient_data['wiek'] = calculate_age(logged_in_patient_data.get('data_urodzenia'))
                return json.dumps({ 'status': 'success', 'message': f"Zalogowano pomyślnie jako {logged_in_patient_data['imie']} {logged_in_patient_data['nazwisko']}." })
            else:
                print(f"Nieudane logowanie dla identyfikatora: {identifier} - błędne hasło.")
                return json.dumps({'status': 'error', 'message': 'Nieprawidłowy identyfikator lub hasło.'})
        else:
            print(f"Nieudane logowanie - nie znaleziono pacjenta dla identyfikatora: {identifier}")
            return json.dumps({'status': 'error', 'message': 'Nieprawidłowy identyfikator lub hasło.'})

    except sqlite3.Error as e:
        print(f"Błąd podczas logowania pacjenta: {e}")
        return json.dumps({'status': 'error', 'message': f'Błąd bazy danych podczas logowania: {e}'})
    finally:
        if conn: conn.close()

try:
    output.register_callback('loginPatientCallback', zaloguj_pacjenta)
    print("Funkcja logowania zarejestrowana.")
except Exception as e:
    print(f"Błąd podczas rejestracji funkcji logowania: {e}")

login_html_content = """
<!DOCTYPE html>
<html lang="pl">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Logowanie Pacjenta</title>
<style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap');
    :root{--primary-blue:#007bff;--light-blue:#e7f3ff;--text-color:#333;--border-color:#ced4da;--background-color:#f8f9fa;--white:#ffffff;--gray:#6c757d;--light-gray:#adb5bd;--link-color:#007bff;--danger-color:#dc3545;--success-color:#28a745;}
    .login-body { background-color: var(--background-color); font-family: 'Roboto', sans-serif; margin: 0; padding: 20px;}
    .login-form-container { background-color: var(--white); max-width: 450px; margin: 40px auto; padding: 30px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); }
    .login-form-container h1 { text-align: center; color: var(--primary-blue); margin-bottom: 30px; font-size: 1.6rem; font-weight: 700;}
    .form-group { margin-bottom: 20px; }
    .form-group label { display: block; margin-bottom: 6px; font-weight: 500; color: var(--text-color); font-size: 0.9rem; }
    .form-group input[type="text"],
    .form-group input[type="password"] { width: 100%; padding: 10px; border: 1px solid var(--border-color); border-radius: 5px; font-size: 1rem; box-sizing: border-box; }
    .form-group input:focus { outline: none; border-color: var(--primary-blue); box-shadow: 0 0 0 2px rgba(0,123,255,0.25); }
    .btn { display: inline-block; width: 100%; padding: 12px; border: none; border-radius: 5px; font-size: 1.1rem; font-weight: 500; cursor: pointer; text-align: center; transition: background-color 0.3s ease, box-shadow 0.3s ease; box-sizing: border-box; background-color: var(--primary-blue); color: var(--white); margin-top: 20px; }
    .btn:disabled { background-color: var(--light-gray); cursor: not-allowed; }
    .btn:hover:not(:disabled) { background-color: #0056b3; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
    #login-status-message { margin-top: 20px; font-weight: 500; min-height: 1.2em; text-align: center; transition: color 0.3s ease, background-color 0.3s ease, border 0.3s ease; padding: 10px; border-radius: 5px; border: 1px solid transparent; }
    .status-success { color: var(--success-color); background-color: #d4edda; border-color: #c3e6cb; }
    .status-error { color: var(--danger-color); background-color: #f8d7da; border-color: #f5c6cb;}
    .status-processing { color: var(--gray); background-color: #e2e3e5; border-color: #d6d8db;}
    .status-info { color: #0c5460; background-color: #d1ecf1; border-color: #bee5eb;} /* Styl dla info */
</style>
</head>
<body class="login-body">

<div class="login-form-container" id="login-form-wrapper">
    <h1>Logowanie Pacjenta</h1>
    <form id="login-patient-form">
        <div class="form-group">
            <label for="login-identifier">Email lub PESEL <span style="color:red;">*</span></label>
            <input type="text" id="login-identifier" name="login-identifier" required>
        </div>
        <div class="form-group">
            <label for="login-password">Hasło <span style="color:red;">*</span></label>
            <input type="password" id="login-password" name="login-password" required>
        </div>
        <button type="submit" id="login-button" class="btn">Zaloguj się</button>
        <div id="login-status-message"></div>
    </form>
</div>

<script>
    const loginForm = document.getElementById('login-patient-form');
    const loginStatusMessageDiv = document.getElementById('login-status-message');
    const loginButton = document.getElementById('login-button');
    const loginIdentifierInput = document.getElementById('login-identifier');
    const loginPasswordInput = document.getElementById('login-password');

    function showLoginStatus(message, type) {
        loginStatusMessageDiv.textContent = message;
        // Użyj klasy status-info dla komunikatu o sprawdzeniu wyniku poniżej
        loginStatusMessageDiv.className = type === 'info' ? 'status-info' : `status-${type}`;
    }

    loginForm.addEventListener('submit', async function(event) {
        event.preventDefault();
        loginButton.disabled = true;
        showLoginStatus('Logowanie...', 'processing');

        const identifier = loginIdentifierInput.value.trim();
        const password = loginPasswordInput.value;

        if (!identifier || !password) {
             showLoginStatus('Email/PESEL i hasło są wymagane.', 'error');
             loginButton.disabled = false;
             return;
        }

        try {
            await google.colab.kernel.invokeFunction(
                'loginPatientCallback',
                [identifier, password],
                {}
            );
            // Zakładamy, że jeśli doszło tutaj, Python się wykonał.
            // Wyświetl komunikat informacyjny, kierujący do wyniku poniżej.
            showLoginStatus('Próba logowania zakończona. Sprawdź wynik poniżej.', 'info');
            loginPasswordInput.value = ''; // Wyczyść hasło

        } catch (error) {
            console.error('Błąd wywołania funkcji Pythona (loginPatient):', error);
            // Błąd komunikacji z backendem
            showLoginStatus('Wystąpił błąd komunikacji z backendem podczas logowania.', 'error');
        } finally {
            loginButton.disabled = false;
        }
    });
</script>

</body>
</html>
"""

print("\nWyświetlanie formularza logowania.")
display(HTML(login_html_content))

Funkcja logowania zarejestrowana.

Wyświetlanie formularza logowania.


Pomyślnie zalogowano pacjenta: test test (ID: 2)


# Karta pacjenta

In [ ]:
# -*- coding: utf-8 -*-
# Blok 3: Wyświetlanie Karty Zalogowanego Pacjenta

from IPython.display import display, HTML, Javascript
import os
from datetime import datetime
import re

patient_card_html = ""
patient_logged_in = 'logged_in_patient_data' in globals() and logged_in_patient_data

if patient_logged_in:
    data = logged_in_patient_data
    print(f"Wyświetlanie danych dla zalogowanego pacjenta: {data.get('imie')} {data.get('nazwisko')}")

    def get(key, defaultValue = 'Brak danych'): return data.get(key) or defaultValue
    def getListItems(text, default_text = 'Brak danych'):
        if not text: return f'<li>{default_text}</li>'
        items = [item.strip() for item in re.split(r'[\n,]+', text) if item.strip()]
        if not items: return f'<li>{default_text}</li>'
        return ''.join([f'<li>{item}</li>' for item in items])
    def formatMultiline(text, default_text = 'Brak danych'):
         if not text: return default_text
         return text.replace('\n', '<br>')

    imie = get('imie', '')
    nazwisko = get('nazwisko', '')
    inicjaly = f'{imie[0]}{nazwisko[0]}'.upper() if (imie and nazwisko) else '??'
    wiek_info = f"({data['wiek']} lat)" if data.get('wiek') is not None else ''

    patient_card_html = f"""
    <!DOCTYPE html>
    <html lang="pl">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Karta Pacjenta - {imie} {nazwisko}</title>
        <style>
            @import url('https://fonts.googleapis.com/css2?family=Roboto:wght@400;500;700&display=swap');
            :root{{--primary-blue:#007bff;--light-blue:#e7f3ff;--text-color:#333;--border-color:#ced4da;--background-color:#f8f9fa;--white:#ffffff;--gray:#6c757d;--light-gray:#adb5bd;--link-color:#007bff;--danger-color:#dc3545;--success-color:#28a745; --warning-color: #ffc107; --info-color: #17a2b8; --alert-bg: #f8d7da; --alert-border: #f5c6cb; --alert-text: #721c24;}}
            body {{ background-color: var(--background-color); font-family: 'Roboto', sans-serif; margin: 0; padding: 20px;}}
            .patient-card-container {{ max-width: 1000px; margin: 30px auto; background-color: var(--white); border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.1); overflow: hidden; }}
            .patient-header {{ background-color: var(--primary-blue); color: var(--white); padding: 25px; display: flex; align-items: center; gap: 20px; flex-wrap: wrap; }}
            .patient-initials {{ background-color: var(--white); color: var(--primary-blue); border-radius: 50%; width: 60px; height: 60px; display: flex; justify-content: center; align-items: center; font-size: 1.8rem; font-weight: bold; flex-shrink: 0; }}
            .patient-info {{ flex-grow: 1; }}
            .patient-info h2 {{ margin: 0 0 5px 0; font-size: 1.6rem; }}
            .patient-info p {{ margin: 0; font-size: 0.9rem; opacity: 0.9; }}
            .patient-status {{ margin-left: auto; background-color: rgba(255, 255, 255, 0.2); padding: 5px 15px; border-radius: 15px; font-size: 0.9rem; order: 3; }}
            .logout-button {{ background-color: var(--danger-color); border: none; color: white; padding: 8px 15px; font-size: 0.9rem; border-radius: 5px; cursor: pointer; order: 4; margin-left: 10px; }}
            .logout-button:hover {{ background-color: #c82333; }}
            .patient-content {{ padding: 25px; display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 25px; }}
            .content-section {{ background-color: var(--white); padding: 20px; border-radius: 6px; border: 1px solid var(--border-color); }}
            .content-section.alert {{ background-color: var(--alert-bg); border-color: var(--alert-border); color: var(--alert-text); }}
            .content-section h3 {{ color: var(--primary-blue); margin-top: 0; margin-bottom: 15px; font-size: 1.1rem; border-bottom: 1px solid var(--border-color); padding-bottom: 8px; }}
            .content-section.alert h3 {{ color: var(--alert-text); border-bottom-color: rgba(0,0,0,0.1); }}
            .content-section p, .content-section ul {{ font-size: 0.95rem; color: var(--text-color); margin: 0 0 10px 0; padding-left: 0; list-style: none;}}
            .content-section ul li {{ margin-bottom: 8px; }}
            .content-section strong {{ font-weight: 500; color: var(--text-color); }}
            .content-section.alert strong {{ color: inherit; }}
            .content-section a {{ color: var(--link-color); text-decoration: none; font-size: 0.9rem; }}
            .content-section a:hover {{ text-decoration: underline; }}
            .result-normal {{ color: var(--success-color); font-weight: bold; }}
            .result-elevated {{ color: var(--danger-color); font-weight: bold; }}
            .quick-actions {{ grid-column: 1 / -1; display: flex; flex-wrap: wrap; gap: 10px; justify-content: flex-start; }}

            /* Ogólne style dla przycisków - powinny już tu być */
            .btn {{
                display: inline-block;
                padding: 8px 15px; /* Mniejszy padding dla przycisków akcji */
                border: none;
                border-radius: 5px;
                font-size: 0.9rem; /* Mniejsza czcionka */
                font-weight: 500;
                cursor: pointer;
                text-align: center;
                transition: background-color 0.3s ease, box-shadow 0.3s ease;
                box-sizing: border-box;
                background-color: var(--primary-blue); /* Główny kolor */
                color: var(--white);
                margin-top: 0; /* Reset marginesu dla przycisków akcji */
                width: auto; /* Szerokość dopasowana do treści */
            }}
            .btn:hover:not(:disabled) {{
                background-color: #0056b3; /* Ciemniejszy niebieski */
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }}
             .btn:disabled {{ /* Styl dla nieaktywnych przycisków */
                 background-color: var(--light-gray);
                 cursor: not-allowed;
             }}

            .not-logged-in {{ text-align: center; margin-top: 50px; font-size: 1.1rem; color: var(--gray); }}
            @media (max-width: 768px) {{
                .patient-header {{ gap: 10px; }}
                .patient-info h2 {{ font-size: 1.4rem; }}
                .patient-info p {{ font-size: 0.85rem; }}
                .patient-status, .logout-button {{ margin-left: 0; margin-top: 10px; width: 100%; text-align: center; }}
                 .logout-button {{ margin-left: 0; }}
                 .quick-actions {{ justify-content: center; }}
             }}
        </style>
    </head>
    <body>
        <div class="patient-card-container" id="patient-card">
            <div class="patient-header">
                <div class="patient-initials">{inicjaly}</div>
                <div class="patient-info">
                    <h2>{imie} {nazwisko}</h2>
                    <p>ID: {get('id')} | PESEL: {get('pesel')} | Ur: {get('data_urodzenia')} {wiek_info}</p>
                </div>
                <div class="patient-status">Aktywny</div>
                 <button class="logout-button" onclick="logoutPatient()">Wyloguj</button>
            </div>
            <div class="patient-content">
                 <div class="content-section alert">
                    <h3>Alerty Medyczne</h3>
                    <ul>
                       {getListItems(get('alergie'), 'Brak alergii')}
                       {getListItems(get('choroby_przewlekle'), 'Brak chorób przewlekłych')}
                    </ul>
                    { f'<p><strong>Ważna uwaga:</strong> {formatMultiline(get("wazne_uwagi"))}</p>' if data.get("wazne_uwagi") else '' }
                </div>
                <div class="content-section">
                    <h3>Dane Osobowe i Kontaktowe</h3>
                    <p><strong>Telefon:</strong> {get('telefon')}</p>
                    <p><strong>Email:</strong> {get('email')}</p>
                    <p><strong>Adres:</strong> {get('adres')}, {get('kod_pocztowy')} {get('miasto')}</p>
                    <p><strong>Kontakt ICE:</strong> {get('kontakt_ice_imie_nazwisko')} ({get('kontakt_ice_relacja')}) - {get('kontakt_ice_telefon')}</p>
                </div>
                <div class="content-section"><h3>Aktywny Plan Leczenia</h3><p>{formatMultiline(get('plan_leczenia', 'Brak aktywnego planu.'))}</p></div>
                <div class="content-section"><h3>Aktywne Skierowania</h3><p>RTG klatki piersiowej (do [Data])</p><p>Konsultacja [Specjalizacja] ([Pilność])</p></div>
                <div class="content-section"><h3>Historia Wizyt</h3><p><strong>Nadchodząca:</strong> [Data], [Godz] (Dr. Kowalski - Kardiolog)</p><p>[Data] (Dr. Nowak - Internista)</p><p><a href="#">Zobacz pełną historię wizyt...</a></p></div>
                <div class="content-section"><h3>Ostatnie Dokumenty</h3><p>Zgoda na zabieg ([Data])</p><p>Zaświadczenie ([Data])</p><p><a href="#">Zobacz wszystkie dokumenty...</a></p></div>
                <div class="content-section"><h3>Ostatnie Wyniki Badań</h3><p>Morfologia ([Data]) - <span class="result-normal">W normie</span></p><p>Glukoza ([Data]) - <span class="result-elevated">Podwyższony</span></p><p><a href="#">Zobacz wszystkie wyniki...</a></p></div>
                <div class="content-section quick-actions"><h3>Szybkie Akcje</h3><button class="btn">Pełna Historia</button><button class="btn">Wyniki Badań</button><button class="btn">Recepty</button><button class="btn">Dokumenty</button><button class="btn">Umów Wizytę</button><button class="btn">Płatności</button><button class="btn">Edytuj Dane</button></div>
            </div>
        </div>
        <script>
            function logoutPatient() {{
                google.colab.kernel.invokeFunction('logoutCallback', [], {{}})
                    .then(() => {{
                        console.log('Wylogowano, ukrywanie karty...');
                        const card = document.getElementById('patient-card');
                        if (card) {{ card.style.display = 'none'; }}
                         if (!document.getElementById('logout-message')) {{
                            const msg = document.createElement('p');
                            msg.id = 'logout-message';
                            msg.className = 'not-logged-in';
                            msg.textContent = 'Zostałeś wylogowany. Uruchom ponownie komórkę logowania, aby się zalogować, a następnie tę komórkę, aby zobaczyć dane.';
                             document.body.appendChild(msg);
                        }} else {{
                            document.getElementById('logout-message').style.display = 'block';
                        }}
                    }})
                    .catch(err => {{
                         console.error('Błąd podczas wylogowywania:', err);
                         alert('Wystąpił błąd podczas wylogowywania.');
                    }});
            }}
        </script>
    </body>
    </html>
    """
else:
    print("Żaden pacjent nie jest aktualnie zalogowany.")
    patient_card_html = """
     <!DOCTYPE html><html><head><title>Brak dostępu</title>
     <style> .not-logged-in { text-align: center; margin-top: 50px; font-size: 1.1rem; color: var(--gray); font-family: 'Roboto', sans-serif;} :root{--gray:#6c757d;} @import url('https://fonts.googleapis.com/css2?family=Roboto&display=swap');</style>
     </head><body>
     <p class="not-logged-in">Proszę się najpierw zalogować w komórce logowania.</p>
     </body></html>
     """

def logout_patient():
    global logged_in_patient_data
    if 'logged_in_patient_data' in globals() and logged_in_patient_data:
        print(f"Wylogowywanie pacjenta: {logged_in_patient_data.get('imie')} {logged_in_patient_data.get('nazwisko')}")
        logged_in_patient_data = None
    else:
        print("Nikt nie był zalogowany.")
    return {}

try:
    if 'logoutCallback' not in output._js_callbacks:
         output.register_callback('logoutCallback', logout_patient)
         print("Funkcja wylogowania zarejestrowana.")
    else:
         print("Funkcja wylogowania już zarejestrowana.")
except Exception as e:
    print(f"Błąd podczas rejestracji funkcji wylogowania: {e}")

display(HTML(patient_card_html))

Wyświetlanie danych dla zalogowanego pacjenta: test test
Błąd podczas rejestracji funkcji wylogowania: module 'google.colab.output' has no attribute '_js_callbacks'
